Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hnodes = 1024

beta1 = .001
beta2 = .005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnodes]))
  biases1 = tf.Variable(tf.zeros([hnodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hnodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(hidden, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  new_loss = loss + beta1 * tf.nn.l2_loss(weights1) + beta2 * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights2) + biases2)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights2) + biases2)

In [6]:
num_steps = 3001

def do_sgd():
    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [7]:
do_sgd()

Initialized
Minibatch loss at step 0: 373.208374
Minibatch accuracy: 9.4%
Validation accuracy: 32.1%
Minibatch loss at step 500: 8.690955
Minibatch accuracy: 77.3%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.918769
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 0.924422
Minibatch accuracy: 85.2%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 0.482348
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 0.542456
Minibatch accuracy: 82.0%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 0.425517
Minibatch accuracy: 85.2%
Validation accuracy: 86.0%
Test accuracy: 92.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
nbatches = train_labels.shape[0] / batch_size
limit_batch = nbatches - 10

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - (batch_size * limit_batch))
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 306.987793
Minibatch accuracy: 7.8%
Validation accuracy: 27.6%
Minibatch loss at step 500: 0.000528
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 0.002233
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.004715
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 0.014190
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 2500: 0.016965
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 0.018546
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Test accuracy: 87.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hnodes]))
  biases1 = tf.Variable(tf.zeros([hnodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([hnodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  logits = tf.matmul(tf.nn.dropout(hidden, .5), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  new_loss = loss + beta1 * tf.nn.l2_loss(weights1) + beta2 * tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights2) + biases2)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights2) + biases2)
    
do_sgd()

Initialized
Minibatch loss at step 0: 446.617126
Minibatch accuracy: 12.5%
Validation accuracy: 28.7%
Minibatch loss at step 500: 31.030727
Minibatch accuracy: 64.8%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 3.275347
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 1.327454
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 0.799311
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.766142
Minibatch accuracy: 77.3%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.533667
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Test accuracy: 91.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Try with network of 3 hidden layers of sizes 1024, 300 and 50 with tanh activations, with dropout on the hidden activations, learning rate decay and early stopping.

In [10]:
batch_size = 128

hnodes1 = 1024
hnodes2 = 300
hnodes3 = 50

beta1 = .01
beta2 = .01
beta3 = .03
beta4 = .05

dropout = True

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  global_step = tf.Variable(0, trainable=False)
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96)
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hnodes1]))
  biases1 = tf.Variable(tf.zeros([hnodes1]))
  weights2 = tf.Variable(tf.truncated_normal([hnodes1, hnodes2]))
  biases2 = tf.Variable(tf.zeros([hnodes2]))
  weights3 = tf.Variable(tf.truncated_normal([hnodes2, hnodes3]))
  biases3 = tf.Variable(tf.zeros([hnodes3]))
  weights4 = tf.Variable(tf.truncated_normal([hnodes3, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.tanh(tf.matmul(tf_train_dataset, weights1) + biases1)
  if dropout:
    hidden2 = tf.nn.tanh(tf.matmul(tf.nn.dropout(hidden1, .5), weights2) + biases2)  
    hidden3 = tf.nn.tanh(tf.matmul(tf.nn.dropout(hidden2, .5), weights3) + biases3)
    logits = tf.matmul(tf.nn.dropout(hidden3, .5), weights4) + biases4
  else:
    hidden2 = tf.nn.tanh(tf.matmul(hidden1, weights2) + biases2)  
    hidden3 = tf.nn.tanh(tf.matmul(hidden2, weights3) + biases3)
    logits = tf.matmul(hidden3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  new_loss = (loss + beta1 * tf.nn.l2_loss(weights1) +
    beta2 * tf.nn.l2_loss(weights2) + beta3 * tf.nn.l2_loss(weights3) +
    beta4 * tf.nn.l2_loss(weights4))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(new_loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_hidden1 = tf.nn.tanh(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_hidden2 = tf.nn.tanh(tf.matmul(valid_hidden1, weights2) + biases2)
  valid_hidden3 = tf.nn.tanh(tf.matmul(valid_hidden2, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden3, weights4) + biases4)
  
  test_hidden1 = tf.nn.tanh(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_hidden2 = tf.nn.tanh(tf.matmul(test_hidden1, weights2) + biases2)
  test_hidden3 = tf.nn.tanh(tf.matmul(test_hidden2, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden3, weights4) + biases4)

In [11]:
num_steps = 3001
with tf.Session(graph=graph) as session:
  init_ops = tf.initialize_all_variables()
  session.run(init_ops)
  print("Initialized")
  valid_accuracy = 0
  for step in range(num_steps):
    params = {
        'weights1': weights1.eval(),
        'weights2': weights2.eval(),
        'weights3': weights3.eval(),
        'weights4': weights4.eval(),
        'biases1': biases1.eval(),
        'biases2': biases2.eval(),
        'biases3': biases3.eval(),
        'biases4': biases4.eval()
    }
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      this_valid_acc = accuracy(valid_prediction.eval(), valid_labels)
      print("Validation accuracy: %.1f%%" % this_valid_acc)
      if this_valid_acc < valid_accuracy and valid_accuracy > 83.4:
        break
      else:
        valid_accuracy = this_valid_acc
  assign_ops = [
    weights1.assign(params['weights1']),
    weights2.assign(params['weights2']),
    weights3.assign(params['weights3']),
    weights4.assign(params['weights4']),
    biases1.assign(params['biases1']),
    biases2.assign(params['biases2']),
    biases3.assign(params['biases3']),
    biases4.assign(params['biases4'])
  ]
  session.run(assign_ops)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.191300
Minibatch accuracy: 14.8%
Validation accuracy: 13.4%
Minibatch loss at step 5: 10.226584
Minibatch accuracy: 14.8%
Validation accuracy: 20.2%
Minibatch loss at step 10: 7.188357
Minibatch accuracy: 10.9%
Validation accuracy: 24.8%
Minibatch loss at step 15: 5.060424
Minibatch accuracy: 24.2%
Validation accuracy: 28.8%
Minibatch loss at step 20: 3.652320
Minibatch accuracy: 24.2%
Validation accuracy: 33.4%
Minibatch loss at step 25: 3.401240
Minibatch accuracy: 11.7%
Validation accuracy: 40.5%
Minibatch loss at step 30: 2.897736
Minibatch accuracy: 14.1%
Validation accuracy: 44.4%
Minibatch loss at step 35: 2.422002
Minibatch accuracy: 15.6%
Validation accuracy: 43.8%
Minibatch loss at step 40: 2.210923
Minibatch accuracy: 17.2%
Validation accuracy: 48.0%
Minibatch loss at step 45: 2.121631
Minibatch accuracy: 25.0%
Validation accuracy: 47.1%
Minibatch loss at step 50: 2.124248
Minibatch accuracy: 23.4%
Validation accuracy: 49.2%
Minibatch